#  Demo Notebook on SQL Server ML Services

This notebook explains how to embed R program within SQL Stored Procedure using SQL Server ML Services

In [ ]:
use COE 

In [ ]:
-- create table to store model output and model name 
--

if exists(select * from sys.tables where [name] = 'vendor_score_models')
    drop table [COE].[dbo].[vendor_score_models] 

create table [COE].[dbo].vendor_score_models
(
    model_name varchar(100),
    vendor_code varchar(100),
    vendor_name varchar(100),
    mat_cat_code varchar(100),
    model varchar(max)

)

In [ ]:
-- embed R script to get score for each vendors
-- 

-- create procedure to run R Script 
--

use COE 
go 

create or alter procedure [dbo].[vendor_scores] 
(
    @model_name varchar(100)
)
as 
begin 


declare @model varchar(max),
            @input_query nvarchar(max),
            @input_script nvarchar(max)

            -- set input data for script 

            set @input_query = N'
            
select vendor_code, vendor_name, material_category_code, acceptedfull, po_item_count, delivery_due, delivery_intime, offer_material_count from [dbo].[vpr_m001]';

-- R Script to obtain vendor scores
-- 

            set @input_script = N'

vendor_score_fn <- function(input_data)
                {
                
                output_data_frame = data.frame()

                input_data$quality <- (input_data$acceptedfull / (input_data$po_item_count - input_data$delivery_due)) * 100
                input_data$delivery <- (input_data$delivery_intime / (input_data$po_item_count - input_data$delivery_due)) * 100
                input_data$price <- (input_data$po_item_count / input_data$offer_material_count) / 100

                               
               
                input_data$supply_variety_1 = round((input_data$po_item_count - min(input_data$po_item_count))/
                                                (max(input_data$po_item_count) - min(input_data$po_item_count)),3)
                input_data$supply_variety_1[is.nan(input_data$supply_variety_1)] <- 0                
                
                input_data$quality[is.nan(input_data$quality)] <- 0
                input_data$quality_1 <- (input_data$quality - min(input_data$quality)) / (max(input_data$quality) - min(input_data$quality))
                input_data$quality_1[is.nan(input_data$quality_1)] <- 0
                              
                input_data$delivery[is.nan(input_data$delivery)] <- 0
                input_data$delivery_1 <- round((input_data$delivery - min(input_data$delivery)) / (max(input_data$delivery) - min(input_data$delivery)),3)
                input_data$delivery_1[is.nan(input_data$delivery_1)] <- 0
                
                input_data$price_1 = round((input_data$price - min(input_data$price)) / (max(input_data$price) - min(input_data$price)),3)
                input_data$price_1[is.nan(input_data$price_1)] <- 0

                
                input_data$supply_variety_2 <- (1/1) * input_data$supply_variety_1

                input_data$quality_2 <- (1/2) * (input_data$supply_variety_1 + input_data$quality_1)

                input_data$delivery_2 <- (1/3) * (input_data$supply_variety_1 + input_data$quality_1 + input_data$delivery_1)

                input_data$price_2 <- (1/4) * (input_data$supply_variety_1 + input_data$quality_1 + input_data$delivery_1 + input_data$price_1)
              
                output_data_frame <- data.frame(supply_variety_final = input_data$supply_variety_2, quality_final = input_data$quality_2, delivery_final = input_data$delivery_2, price_final = input_data$price_2)
                output_data_frame$vendor_scores <- apply(output_data_frame, 1, max)
                output_data_frame <- cbind(input_data$vendor_code, input_data$vendor_name, input_data$material_category_code, output_data_frame)
                colnames(output_data_frame) <- c("vendor_code", "vendor_name", "material_category_code", "score")
                output_data_frame <- subset(output_data_frame, select = c(vendor_code, vendor_name, material_category_code, score))
                              
                return(output_data_frame)

    }  

    scores <- vendor_score_fn(input_data = indata)

    OutputDataSet <- data.frame(model_name = model_name, vendor_code = scores$vendor_code, vendor_name = scores$vendor_name, mat_cat_code = scores$material_category_code, model = scores$score)

    ';

            --- save script to table 
            delete from vendor_score_models where model_name = @model_name;
            insert into vendor_score_models (model_name, vendor_code, vendor_name, mat_cat_code, model) 

            --- execute external R script 
            ---

            execute sp_execute_external_script 

            @language = N'R',
            @script = @input_script,
            @input_data_1 = @input_query,
            @input_data_1_name = N'indata',
            @params = N'@model_name varchar(100)',
            @model_name = @model_name;

end;
go 


In [ ]:
-- step 1
--

use COE 
go

declare @model_name varchar(100) = 'vendor model (R)'
EXECUTE [dbo].[vendor_scores] @model_name

-- view top 10 rows 
-- 
select top 10 * from dbo.vendor_score_models where model_name = @model_name order by vendor_code